In [ ]:
!git clone https://github.com/dudegladiator/ResumeParser

In [ ]:
!pip install spacy==3.7.2

In [ ]:
from spacy.tokens import DocBin
from tqdm import tqdm
import random
import json
import spacy
from spacy.util import filter_spans
nlp = spacy.blank("en") # load a new spacy model
doc_bin = DocBin()


!python -m spacy download en_core_web_lg

In [ ]:

def convert_dataturks_to_spacy(filepath, testing_data_proportion=0.2):
    random.seed(0)
    try:
        all_data = []
        with open(filepath, 'r') as f:
            lines = f.readlines()

        for line in lines:
            line = line.lower()
            data = json.loads(line)
            text = data['content']
            entities = []
            data_annotations = data['annotation']
            if data_annotations is not None:
                for annotation in data_annotations:
                    # only a single point in text annotation.
                    point = annotation['points'][0]
                    labels = annotation['label']
                    # handle both list of labels or a single label.
                    if not isinstance(labels, list):
                        labels = [labels]

                    for label in labels:
                        point_start = point['start']
                        point_end = point['end']
                        point_text = point['text']

                        lstrip_diff = len(point_text) - len(point_text.lstrip())
                        rstrip_diff = len(point_text) - len(point_text.rstrip())
                        if lstrip_diff != 0:
                            point_start = point_start + lstrip_diff
                        if rstrip_diff != 0:
                            point_end = point_end - rstrip_diff
                        entities.append((point_start, point_end + 1, label))
            all_data.append((text, {"entities": entities}))
        training_data = [data for data in all_data ]
        return training_data

    except Exception as e:
        print("Error occurred: ", str(e))

In [ ]:
#start with harini dataset
train_data = convert_dataturks_to_spacy("/content/model3_dataset.json")

In [ ]:
training_data = []
for example in train_data:
  temp_dict = {}
  temp_dict['text'] = example[0]
  temp_dict['entities'] = []
  for annotation in example[1]["entities"]:
    if annotation[2] == '' or annotation[2] is None or annotation[2] == 'UNKNOWN':
      label = 'ignore'
    start = annotation[0]
    end = annotation[1]
    label = annotation[2].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)

print(training_data[0])

In [ ]:
count=0
for training_example  in tqdm(training_data):
    text = training_example['text']
    labels = training_example['entities']
    doc = nlp.make_doc(text)
    ents = []
    for start, end, label in labels:
        if label == 'ignore':
          continue
        span = doc.char_span(start, end, label=label, alignment_mode="expand")
        if span is None:
            print("Skipping entity")
            print(start, end, label)
            count+=1
        else:
            ents.append(span)
    filtered_ents = filter_spans(ents)
    doc.ents = filtered_ents
    doc_bin.add(doc)
print(count)
doc_bin.to_disk("model3.spacy")

Please get the base config file from spacy website

In [ ]:
!python -m spacy init fill-config base_config.cfg config.cfg

If you get the config file, then directly run below code

---

Upload the config file to root


In [ ]:
!python -m spacy train config.cfg --output ./output3 --paths.train ./model3.spacy --paths.dev ./model3.spacy

go to the ouput3 folder, model-best is model3

In [ ]:
model3 = spacy.load("/content/output3/model-best")

In [ ]:
model3.get_pipe("ner").labels